In [103]:
# -------------------------------------
# CS6320 - NLP - Assignment 1 - Bigram Model
# -------------------------------------

print("\n-------------------------\nGroup 7 - NLP Assignment 1")
print("Bhanu Maneesh Reddy Mannem (BXM220055)")
print("Snehal Kumar Ketala (SXK220463)")
print("Lalithya Mada (LXM230002)")
print("-------------------------")


-------------------------
Group 7 - NLP Assignment 1
Bhanu Maneesh Reddy Mannem (BXM220055)
Snehal Kumar Ketala (SXK220463)
Lalithya Mada (LXM230002)
-------------------------


In [104]:
import requests
import re
import math
from collections import defaultdict

 **Data Download and Preprocessing**

In [105]:
def download_dataset():
    train_url = "https://raw.githubusercontent.com/maneeshmbr/unigram-and-bigram-models---NLP/master/A1_DATASET/A1_DATASET/train.txt"
    val_url = "https://raw.githubusercontent.com/maneeshmbr/unigram-and-bigram-models---NLP/master/A1_DATASET/A1_DATASET/val.txt"

    train_data = requests.get(train_url).text
    with open('training_data.txt', 'w') as train_file:
        train_file.write(train_data)

    val_data = requests.get(val_url).text
    with open('validation_data.txt', 'w') as val_file:
        val_file.write(val_data)

def preprocess_text(text):
    clean_text = re.sub(r'\W', ' ', text).lower()
    tokens = clean_text.split()
    bigrams = [(tokens[i], tokens[i+1]) for i in range(len(tokens) - 1)]
    return tokens, bigrams

def load_and_preprocess(file_path):
    with open(file_path, 'r') as file:
        text = file.read()

    tokens, bigrams = preprocess_text(text)
    return tokens, bigrams

download_dataset()
train_tokens, train_bigrams = load_and_preprocess('training_data.txt')
val_tokens, val_bigrams = load_and_preprocess('validation_data.txt')


**Unsmoothed Bigram Calculation and its probability**

In [106]:
def calculate_bigram_probabilities(tokens, bigrams):
    unigram_counts = defaultdict(int)
    bigram_counts = defaultdict(int)

    for token in tokens:
        unigram_counts[token] += 1

    for bigram in bigrams:
        bigram_counts[bigram] += 1

    bigram_probs = {bigram: bigram_counts[bigram] / unigram_counts[bigram[0]] for bigram in bigram_counts}
    bigram_log_probs = {bigram: math.log(bigram_probs[bigram]) for bigram in bigram_probs}

    return bigram_probs, bigram_log_probs, unigram_counts, bigram_counts

bigram_probs, bigram_log_probs, unigram_counts, bigram_counts = calculate_bigram_probabilities(train_tokens, train_bigrams)


**Perplexity Calculation**

In [107]:
def calculate_perplexity(bigram_probs, val_bigrams, log_prob_dict):
    val_bigram_counts = defaultdict(int)
    token_count = 0
    for bigram in val_bigrams:
        val_bigram_counts[bigram] += 1
        token_count += 1

    log_prob_sum = 0
    for bigram in val_bigram_counts:
        log_prob_sum += (-1) * log_prob_dict.get(bigram, log_prob_dict.get(('unk', 'unk'), 0)) * val_bigram_counts[bigram]

    return math.exp(log_prob_sum / token_count)


**Smoothing (Laplace and Add-k)**

In [108]:
def laplace_smoothing(bigram_counts, unigram_counts, vocab_size):
    laplace_probs = {}
    laplace_log_probs = {}

    for bigram in bigram_counts:
        laplace_probs[bigram] = (bigram_counts[bigram] + 1) / (unigram_counts[bigram[0]] + vocab_size)
        laplace_log_probs[bigram] = math.log(laplace_probs[bigram])

    return laplace_probs, laplace_log_probs

def add_k_smoothing(bigram_counts, unigram_counts, k, vocab_size):
    add_k_probs = {}
    add_k_log_probs = {}

    for bigram in bigram_counts:
        add_k_probs[bigram] = (bigram_counts[bigram] + k) / (unigram_counts[bigram[0]] + k * vocab_size)
        add_k_log_probs[bigram] = math.log(add_k_probs[bigram])

    return add_k_probs, add_k_log_probs


**Handling Unknown Words (Unk) and Rare Words**

In [109]:
def handle_unknown_with_zero_frequency(tokens):
    """
    Replace words that occur only once with 'unk' and keeps 'unk' with zero frequency.
    """
    unigram_counts = defaultdict(int)
    for token in tokens:
        unigram_counts[token] += 1

    # Add 'unk' with zero frequency
    unigram_counts['unk'] = 0

    tokens_unk = ['unk' if unigram_counts[token] == 1 else token for token in tokens]
    return tokens_unk, unigram_counts

def handle_rare_words(tokens, rare_threshold=2):
    """
    Replace words that occur less than or equal to the threshold with 'unk'.
    """
    word_counts = defaultdict(int)
    for token in tokens:
        word_counts[token] += 1

    tokens_rare_unk = ['unk' if word_counts[token] <= rare_threshold else token for token in tokens]
    return tokens_rare_unk

# Handle unknown words with zero frequency
train_tokens_unk, unigram_counts_unk = handle_unknown_with_zero_frequency(train_tokens)
# Handle rare words with a threshold of 2
train_tokens_rare_unk = handle_rare_words(train_tokens)


**Perplexity Calculations for All Scenarios**

In [110]:
def calculate_all_perplexities():
    results = {}
    vocab_size = len(unigram_counts)

    # 1. Perplexity with No Smoothing
    perplexity_no_smoothing = calculate_perplexity(bigram_probs, val_bigrams, bigram_log_probs)
    results['Perplexity - No Smoothing'] = perplexity_no_smoothing

    # 2. Add-k Smoothing and Laplace Smoothing
    k_values = [0.5, 3]
    laplace_probs, laplace_log_probs = laplace_smoothing(bigram_counts, unigram_counts, vocab_size)
    perplexity_laplace = calculate_perplexity(laplace_probs, val_bigrams, laplace_log_probs)
    results['Perplexity - Laplace Smoothing'] = perplexity_laplace

    for k in k_values:
        add_k_probs, add_k_log_probs = add_k_smoothing(bigram_counts, unigram_counts, k, vocab_size)
        perplexity_add_k = calculate_perplexity(add_k_probs, val_bigrams, add_k_log_probs)
        results[f'Perplexity - Add-k Smoothing (k={k})'] = perplexity_add_k

    # 3. Add-k Smoothing and Laplace Smoothing with 'unk' (zero frequency)
    train_bigrams_unk = [(train_tokens_unk[i], train_tokens_unk[i + 1]) for i in range(len(train_tokens_unk) - 1)]
    bigram_probs_unk, bigram_log_probs_unk, unigram_counts_unk, bigram_counts_unk = calculate_bigram_probabilities(train_tokens_unk, train_bigrams_unk)

    laplace_unk_probs, laplace_unk_log_probs = laplace_smoothing(bigram_counts_unk, unigram_counts_unk, len(unigram_counts_unk))
    perplexity_laplace_unk = calculate_perplexity(laplace_unk_probs, val_bigrams, laplace_unk_log_probs)
    results['Perplexity - Laplace Smoothing (unk)'] = perplexity_laplace_unk

    for k in k_values:
        add_k_unk_probs, add_k_unk_log_probs = add_k_smoothing(bigram_counts_unk, unigram_counts_unk, k, len(unigram_counts_unk))
        perplexity_add_k_unk = calculate_perplexity(add_k_unk_probs, val_bigrams, add_k_unk_log_probs)
        results[f'Perplexity - Add-k Smoothing (unk, k={k})'] = perplexity_add_k_unk

    # 4. Add-k Smoothing and Laplace Smoothing with rare words as 'unk'
    train_bigrams_rare_unk = [(train_tokens_rare_unk[i], train_tokens_rare_unk[i + 1]) for i in range(len(train_tokens_rare_unk) - 1)]
    bigram_probs_rare, bigram_log_probs_rare, unigram_counts_rare, bigram_counts_rare = calculate_bigram_probabilities(train_tokens_rare_unk, train_bigrams_rare_unk)

    laplace_rare_probs, laplace_rare_log_probs = laplace_smoothing(bigram_counts_rare, unigram_counts_rare, len(unigram_counts_rare))
    perplexity_laplace_rare = calculate_perplexity(laplace_rare_probs, val_bigrams, laplace_rare_log_probs)
    results['Perplexity - Laplace Smoothing (rare words)'] = perplexity_laplace_rare

    for k in k_values:
        add_k_rare_probs, add_k_rare_log_probs = add_k_smoothing(bigram_counts_rare, unigram_counts_rare, k, len(unigram_counts_rare))
        perplexity_add_k_rare = calculate_perplexity(add_k_rare_probs, val_bigrams, add_k_rare_log_probs)
        results[f'Perplexity - Add-k Smoothing (rare words, k={k})'] = perplexity_add_k_rare

    return results


**Display Results**

In [111]:
def display_results(results):
    print("Perplexity Results:")
    for scenario, perplexity in results.items():
        print(f"{scenario}: {perplexity}")

# Run all perplexity calculations
results = calculate_all_perplexities()

# Display all the results
display_results(results)

Perplexity Results:
Perplexity - No Smoothing: 9.593040871653818
Perplexity - Laplace Smoothing: 60.17091971583495
Perplexity - Add-k Smoothing (k=0.5): 43.7558536740361
Perplexity - Add-k Smoothing (k=3): 97.7601456201918
Perplexity - Laplace Smoothing (unk): 135.37235609080957
Perplexity - Add-k Smoothing (unk, k=0.5): 91.95803128753613
Perplexity - Add-k Smoothing (unk, k=3): 270.69792197096535
Perplexity - Laplace Smoothing (rare words): 88.98504438081802
Perplexity - Add-k Smoothing (rare words, k=0.5): 64.32474776593875
Perplexity - Add-k Smoothing (rare words, k=3): 162.17594532867278
